# ⭐ Shopee - Price Match Guarantee
- Determine if two products are the same by their images
- https://www.kaggle.com/competitions/shopee-product-matching/overview

### Shopee - Price Match Guarantee: 문제 개요 및 태스크 요약

---

#### 태스크 목적

- 두 제품이 동일한 상품인지 **이미지와 텍스트** 정보를 활용해 자동으로 판단하는 멀티모달 제품 매칭 문제입니다.
- 동일 제품임에도 사진, 제목 등이 다를 수 있어 단순 비교가 어렵고, 이를 딥러닝 기반으로 해결하는 것이 목표입니다.

---

#### 데이터 및 문제 특성

- 각 제품은 `image` (사진)와 `title` (상품명) 텍스트를 가집니다.
- 제품들은 `label_group`으로 묶여 있으며, 같은 그룹은 같은 제품을 의미합니다.
- 제품 쌍(positive/negative)을 만들어 **두 제품이 같은 그룹인지 여부**를 이진 분류합니다.

---

#### 구현한 코드의 주요 변경 및 특징

- **멀티모달 입력**: 이미지와 텍스트를 모두 입력으로 받는 Siamese 네트워크 구조를 사용  
  → 각 쌍에서 두 제품의 이미지 임베딩과 텍스트 임베딩을 CLIP 모델로 추출하여 결합

- **그룹 단위로 train/val/test 분할**  
  → 같은 그룹에 속한 제품들은 같은 split에 배정하여 정보 누출 방지

- **Positive / Negative 쌍 생성**  
  → 같은 그룹에서 조합 가능한 제품 쌍은 positive (1)  
  → 다른 그룹 간 제품 쌍은 negative (0), negative 쌍은 positive 쌍 대비 최대 2배 비율로 생성

- **학습 목표**:  
  - 입력: 두 제품의 이미지와 텍스트  
  - 출력: 두 제품이 같은 상품인지 여부 (확률)  
  - 손실 함수: `BCEWithLogitsLoss`를 사용하여 이진 분류 학습

- **평가**:  
  - 테스트셋에서 정확도와 손실을 계산  
  - 실제 대회 평가 지표는 mean F1 score이나, 코드는 정확도를 기준으로 평가

---

#### 기대 효과 및 활용처

- 제품 매칭 자동화로 잘못된 상품 중복이나 오인 문제 감소  
- 쇼핑 플랫폼에서 정확한 상품 분류와 가격 비교 지원  
- 소비자는 최적의 가격과 상품 정보를 더 쉽게 찾을 수 있음

---

#### 참고

- Shopee의 주요 동남아시아 및 대만 전자상거래 플랫폼  
- 문제는 2021년 Kaggle 대회 `Shopee Product Matching` 기반

---


# ⭐ 멀티모달 모델 (이미지 + 텍스트)

### 모델 개요: 이미지+텍스트를 모두 활용하는 멀티모달 Siamese 모델

---

#### 1. 입력 (Input)

- 한 쌍(pair)의 데이터가 들어갑니다.
- 각 데이터는 **이미지 + 텍스트**로 구성되어 있습니다.
- 구체적으로 입력은 다음과 같습니다:
  - `input_ids1`, `attention_mask1`, `pixel_values1`: 첫 번째 샘플의 텍스트 토큰, 어텐션 마스크, 이미지 픽셀값
  - `input_ids2`, `attention_mask2`, `pixel_values2`: 두 번째 샘플의 텍스트 토큰, 어텐션 마스크, 이미지 픽셀값

---

#### 2. 모델 내부 처리

- CLIP 모델을 사용하여 텍스트와 이미지를 각각 임베딩합니다.
  - `get_text_features()` → 텍스트 임베딩 벡터 (예: 512차원)
  - `get_image_features()` → 이미지 임베딩 벡터 (예: 512차원)
- 각 샘플에서 이미지 임베딩과 텍스트 임베딩을 연결(concatenate)하여 하나의 벡터로 만듭니다.
- 두 샘플의 벡터를 다시 연결(concatenate)하여 `(projection_dim * 4)` 차원의 벡터를 생성합니다.
- 이 벡터를 분류기(classifier)에 통과시켜 두 샘플의 유사성 점수를 출력합니다.

---

#### 3. 출력 (Output)

- 모델의 출력은 두 샘플이 같은 그룹인지 아닌지에 대한 **로짓(logit)** 값입니다.
- 이 값을 시그모이드 함수에 통과시켜 확률로 변환할 수 있습니다.
- 확률이 0.5 이상이면 "같은 그룹(positive pair)"으로 예측합니다.

---

#### 요약

| 구분      | 내용                                           |
| --------- | ---------------------------------------------- |
| 입력      | 두 샘플 (이미지1 + 텍스트1, 이미지2 + 텍스트2) |
| 내부 처리 | CLIP 모델로 이미지와 텍스트 각각 임베딩 후 concat |
| 출력      | 두 샘플의 유사성 점수 (로짓, 확률로 변환 가능)   |

---

이 모델은 이미지와 텍스트 두 정보를 모두 활용하여 제품 쌍의 유사성을 판단하는 **멀티모달 쌍별 분류기**입니다.


In [1]:
# 0. 라이브러리 임포트
import os, random, datetime
import pandas as pd
import numpy as np
from PIL import Image
from itertools import combinations
from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import CLIPProcessor, CLIPModel

# 1. 설정
DATA_DIR = r"D:\Project\PJT_10\shopee-product-matching"
CSV_PATH = os.path.join(DATA_DIR, "train.csv")
IMG_DIR = os.path.join(DATA_DIR, "train_images")

SAVE_DIR = "./saved_models"
os.makedirs(SAVE_DIR, exist_ok=True)

BATCH_SIZE = 32
EPOCHS = 10
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if DEVICE.type == "cuda":
        torch.cuda.manual_seed_all(seed)
set_seed(SEED)

# 2. 데이터 로딩 및 라벨 인코딩
df = pd.read_csv(CSV_PATH).reset_index(drop=True)
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label_group"])

# 3. 그룹 단위로 train/val/test split
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=1, test_size=0.4, random_state=SEED)
train_idx, temp_idx = next(gss.split(df, groups=df["label_encoded"]))

train_df = df.iloc[train_idx].reset_index(drop=True)
temp_df = df.iloc[temp_idx].reset_index(drop=True)

gss2 = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=SEED)
val_idx, test_idx = next(gss2.split(temp_df, groups=temp_df["label_encoded"]))

val_df = temp_df.iloc[val_idx].reset_index(drop=True)
test_df = temp_df.iloc[test_idx].reset_index(drop=True)

print(f"Train size: {len(train_df)}, Val size: {len(val_df)}, Test size: {len(test_df)}")

# 4. Positive / Negative pair 생성 함수
def create_pairs(df, max_neg_per_pos=2):
    pairs = []
    label_groups = df["label_encoded"].unique()

    for lg in label_groups:
        group_df = df[df["label_encoded"] == lg]
        if len(group_df) < 2:
            continue
        idxs = group_df.index.tolist()
        pos_combs = list(combinations(idxs, 2))
        for i, j in pos_combs:
            pairs.append((i, j, 1))

    pos_count = sum(1 for _,_,label in pairs if label == 1)
    neg_needed = pos_count * max_neg_per_pos

    all_indices = list(df.index)  # <- 여기 변경
    neg_pairs = set()

    while len(neg_pairs) < neg_needed:
        i, j = random.sample(all_indices, 2)
        if df.loc[i, "label_encoded"] != df.loc[j, "label_encoded"]:
            neg_pairs.add((i, j))
    for i, j in neg_pairs:
        pairs.append((i, j, 0))

    return pairs

# 5. Dataset 클래스
class ShopeePairDataset(Dataset):
    def __init__(self, df, pairs, img_dir, processor):
        self.df = df
        self.pairs = pairs
        self.img_dir = img_dir
        self.processor = processor

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        i1, i2, label = self.pairs[idx]
        row1 = self.df.loc[i1]
        row2 = self.df.loc[i2]

        image1 = Image.open(os.path.join(self.img_dir, row1["image"])).convert("RGB")
        image2 = Image.open(os.path.join(self.img_dir, row2["image"])).convert("RGB")

        text1 = row1["title"]
        text2 = row2["title"]

        return {"image1": image1, "text1": text1,
                "image2": image2, "text2": text2,
                "label": label}

# 6. Collate 함수
def collate_fn(batch):
    texts1 = [item["text1"] for item in batch]
    texts2 = [item["text2"] for item in batch]
    images1 = [item["image1"] for item in batch]
    images2 = [item["image2"] for item in batch]
    labels = torch.tensor([item["label"] for item in batch], dtype=torch.float)

    inputs1 = processor(text=texts1, images=images1, return_tensors="pt", padding=True, truncation=True)
    inputs2 = processor(text=texts2, images=images2, return_tensors="pt", padding=True, truncation=True)

    return {
        "input_ids1": inputs1["input_ids"],
        "attention_mask1": inputs1["attention_mask"],
        "pixel_values1": inputs1["pixel_values"],

        "input_ids2": inputs2["input_ids"],
        "attention_mask2": inputs2["attention_mask"],
        "pixel_values2": inputs2["pixel_values"],

        "label": labels
    }

# 7. 모델 정의 (Siamese)
class CLIPSiameseModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.clip = CLIPModel.from_pretrained(model_name)
        # 모델 마지막 레이어
        self.classifier = nn.Sequential(
            nn.Linear(self.clip.config.projection_dim * 4, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, input_ids1, attention_mask1, pixel_values1,
                      input_ids2, attention_mask2, pixel_values2):
        text_features1 = self.clip.get_text_features(input_ids=input_ids1, attention_mask=attention_mask1)
        image_features1 = self.clip.get_image_features(pixel_values=pixel_values1)
        feat1 = torch.cat([image_features1, text_features1], dim=1)

        text_features2 = self.clip.get_text_features(input_ids=input_ids2, attention_mask=attention_mask2)
        image_features2 = self.clip.get_image_features(pixel_values=pixel_values2)
        feat2 = torch.cat([image_features2, text_features2], dim=1)

        combined = torch.cat([feat1, feat2], dim=1)
        output = self.classifier(combined).squeeze(1)
        return output

# 8. 데이터셋 및 데이터로더 준비
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

train_pairs = create_pairs(train_df)
val_pairs = create_pairs(val_df)
test_pairs = create_pairs(test_df)

train_dataset = ShopeePairDataset(train_df, train_pairs, IMG_DIR, processor)
val_dataset = ShopeePairDataset(val_df, val_pairs, IMG_DIR, processor)
test_dataset = ShopeePairDataset(test_df, test_pairs, IMG_DIR, processor)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# 9. 학습 루프 및 평가
model = CLIPSiameseModel("openai/clip-vit-base-patch32").to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

best_val_loss = float('inf')
log_list = []

patience = 3  # 개선 없을 때 최대 허용 epoch 수
counter = 0   # 얼리 스탑 카운터

for epoch in range(EPOCHS):
    model.train()
    
    train_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(train_loader, desc=f"Train Epoch {epoch+1}"):
        optimizer.zero_grad()
        inputs = {k: v.to(DEVICE) for k, v in batch.items() if k != "label"}
        labels = batch["label"].to(DEVICE)

        outputs = model(**inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        probs = torch.sigmoid(outputs)        # logits → 확률
        preds = (probs >= 0.5).float()        # threshold 적용
        
        train_loss += loss.item() * labels.size(0)
        
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss /= len(train_loader.dataset)
    train_acc = correct / total

    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}"):
            inputs = {k: v.to(DEVICE) for k, v in batch.items() if k != "label"}
            labels = batch["label"].to(DEVICE)

            outputs = model(**inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).float()
            
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_acc = correct / total

    print(f"Epoch {epoch+1} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    log_list.append({
        "epoch": epoch+1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = os.path.join(SAVE_DIR, f"clip_pair_best_epoch{epoch+1}_{timestamp}.pth")
        torch.save(model.state_dict(), save_path)
        print(f"✅ Saved best model at epoch {epoch+1} to {save_path}")

log_df = pd.DataFrame(log_list)
log_csv_path = os.path.join(SAVE_DIR, f"training_log_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv")
log_df.to_csv(log_csv_path, index=False)
print(f"\n📊 Training log saved to {log_csv_path}")
    
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0

    criterion = nn.BCEWithLogitsLoss()
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            inputs = {k: v.to(DEVICE) for k, v in batch.items() if k != "label"}
            labels = batch["label"].to(DEVICE)

            outputs = model(**inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * labels.size(0)
            
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).float()
            
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    avg_loss = total_loss / total
    
    print(f"\n🧪 Test Loss: {avg_loss:.4f} | Test Accuracy: {acc:.4f}")

    # 모델 파일명 기반 저장 이름 생성
    model_name = os.path.splitext(os.path.basename(best_model_path))[0]  # 'clip_pair_best_epoch3_20250715_235911'
    result_path = os.path.join(SAVE_DIR, f"{model_name}_test_result.txt")

    # 결과 파일 저장
    with open(result_path, "w") as f:
        f.write(f"Test Loss: {avg_loss:.4f}\n")
        f.write(f"Test Accuracy: {acc:.4f}\n")

    print(f"📝 Test results saved to {result_path}")


best_models = sorted([f for f in os.listdir(SAVE_DIR) if f.startswith("clip_pair_best") and f.endswith(".pth")])
best_model_path = os.path.join(SAVE_DIR, best_models[-1])
model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
print(f"✅ Loaded best model from {best_model_path}")

evaluate(model, test_loader)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Train size: 20392, Val size: 6820, Test size: 7038


Validation Epoch 1: 100%|█████████████████████████████████| 1456/1456 [15:29<00:00,  1.57it/s]


Epoch 1 | Train Loss: 0.2741 | Train Acc: 0.8852 | Val Loss: 0.3011 | Val Acc: 0.8963
✅ Saved best model at epoch 1 to ./saved_models\clip_pair_best_epoch1_20250716_225154.pth


Validation Epoch 2: 100%|█████████████████████████████████| 1456/1456 [15:27<00:00,  1.57it/s]


...

Epoch 9 | Train Loss: 0.0307 | Train Acc: 0.9904 | Val Loss: 0.5506 | Val Acc: 0.9012


Validation Epoch 10: 100%|████████████████████████████████| 1456/1456 [15:28<00:00,  1.57it/s]


Epoch 10 | Train Loss: 0.0282 | Train Acc: 0.9913 | Val Loss: 0.6696 | Val Acc: 0.9000

📊 Training log saved to ./saved_models\training_log_20250717_120149.csv
✅ Loaded best model from ./saved_models\clip_pair_best_epoch2_20250717_001936.pth


Evaluating: 100%|█████████████████████████████████████████| 1741/1741 [18:22<00:00,  1.58it/s]


🧪 Test Loss: 0.3174 | Test Accuracy: 0.9016
📝 Test results saved to ./saved_models\clip_pair_best_epoch2_20250717_001936_test_result.txt


In [2]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0

    criterion = nn.BCEWithLogitsLoss()
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            inputs = {k: v.to(DEVICE) for k, v in batch.items() if k != "label"}
            labels = batch["label"].to(DEVICE)

            outputs = model(**inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * labels.size(0)
            
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).float()
            
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    avg_loss = total_loss / total
    
    print(f"\n🧪 Test Loss: {avg_loss:.4f} | Test Accuracy: {acc:.4f}")

    # 모델 파일명 기반 저장 이름 생성
    model_name = os.path.splitext(os.path.basename(best_model_path))[0]  # 'clip_pair_best_epoch3_20250715_235911'
    result_path = os.path.join(SAVE_DIR, f"{model_name}_test_result.txt")

    # 결과 파일 저장
    with open(result_path, "w") as f:
        f.write(f"Test Loss: {avg_loss:.4f}\n")
        f.write(f"Test Accuracy: {acc:.4f}\n")

    print(f"📝 Test results saved to {result_path}")

best_models = sorted([f for f in os.listdir(SAVE_DIR) if f.startswith("clip_pair_best") and f.endswith(".pth")])
best_model_path = os.path.join(SAVE_DIR, best_models[-1])
model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
evaluate(model, test_loader)

from sklearn.metrics import f1_score
import torch
from tqdm import tqdm

model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Computing F1"):
        inputs = {k: v.to(DEVICE) for k, v in batch.items() if k != "label"}
        labels = batch["label"].to(DEVICE)

        outputs = model(**inputs)
        probs = torch.sigmoid(outputs)
        preds = (probs >= 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# f1-score 계산 (이진 분류라면 'binary')
f1 = f1_score(all_labels, all_preds, average='binary')
print(f"✅ Test F1-score: {f1:.4f}")

Evaluating: 100%|█████████████████████████████████████████| 1741/1741 [18:21<00:00,  1.58it/s]



🧪 Test Loss: 0.3174 | Test Accuracy: 0.9016
📝 Test results saved to ./saved_models\clip_pair_best_epoch2_20250717_001936_test_result.txt


Computing F1: 100%|███████████████████████████████████████| 1741/1741 [18:23<00:00,  1.58it/s]

✅ Test F1-score: 0.8447
